In [3]:
import scanpy as sc
from omnicell.data.loader import DataLoader, DatasetDetails
import torch 
from transformers import AutoTokenizer, AutoModel
import transformers

print(torch.cuda.is_available())


True


In [4]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(
    'chaoyi-wu/PMC_LLAMA_7B',
    local_files_only=False
    )

model = transformers.LlamaForCausalLM.from_pretrained(
    'chaoyi-wu/PMC_LLAMA_7B',
    local_files_only=False
).to("cuda")

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


/orcd/data/omarabu/001/opitcho/miniforge3/envs/huggingface/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `pad_token_id` should be positive but got -1. This will cause errors when batch generating, if there is padding. Please set `pad_token_id` explicitly as `model.generation_config.pad_token_id=PAD_TOKEN_ID` to avoid errors in generation
  warnings.warn(
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.60it/s]


In [5]:
from omnicell.data.catalogue import Catalogue, DatasetDetails
import numpy as np
catalogue = Catalogue("./configs/catalogue")

#Getting the dataset details from the data_catalogue.json

ds_details = catalogue.get_dataset_details("satija_IFNB_raw")
pert_key = ds_details.pert_key
control_pert = ds_details.control


adata = sc.read(ds_details.path, backed ="r+")

gene_names = adata.var["gene"]


gene_names_idx = gene_names.index.to_numpy().astype(np.int32) - 1

gene_names = list(gene_names)
tokenizer.pad_token = tokenizer.eos_token

gene_names_idx

BlockingIOError: [Errno 11] Unable to synchronously open file (unable to lock file, errno = 11, error message = 'Resource temporarily unavailable')

In [ ]:
batch_size = 32 
tokenized_batches = []
for i in range(0, len(gene_names), batch_size):
    indices = gene_names_idx[i:i+batch_size]
    genes = gene_names[i:i+batch_size]
    tokenized_gene_names_batch = tokenizer(genes, return_tensors="pt", padding=True, truncation=True, max_length=30)
    tokenized_gene_names_batch["idx"] = indices
    tokenized_batches.append(tokenized_gene_names_batch)


In [ ]:
import gc

#Clearing cuda cache and gc to free up memory
torch.cuda.empty_cache()
gc.collect()

0

In [ ]:
gene_representations = torch.zeros((len(gene_names), 32000))

for batch in tokenized_batches:
    # Move batch to GPU
    batch_gpu = {k: v.to("cuda") for k, v in batch.items() if k != "idx"}
    
    # Forward pass
    with torch.no_grad():  # Prevent gradient computation if not needed
        outputs = model(**batch_gpu)
    
    # Move results back to CPU immediately
    mean_logits = torch.mean(outputs.logits, dim=1).cpu()
    
    # Store results
    gene_representations[batch["idx"]] = mean_logits
    
    # Clear GPU cache periodically
    if batch["idx"][0] % 100 == 0:  # Every 100 batches
        torch.cuda.empty_cache()
        
    # Delete unnecessary tensors
    del outputs, batch_gpu



In [ ]:
torch.save({"repr" : gene_representations, "gene_names" : gene_names} , "notebooks/gene_representations_llamaPMC.pt")

In [ ]:
gene_representations.shape

torch.Size([34025, 32000])

In [ ]:
import torch
import umap
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# Convert to numpy and standardize
data = gene_representations.cpu().numpy()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)

# PCA first - reduce to 100 dimensions
print("Running PCA...")
pca = PCA(n_components=100)
data_pca = pca.fit_transform(data_scaled)

# Calculate variance explained
var_explained = pca.explained_variance_ratio_.cumsum()
print(f"Variance explained by 100 PCs: {var_explained[-1]:.3f}")

# UMAP on PCA results
print("Running UMAP...")
reducer = umap.UMAP(
    n_neighbors=30,
    min_dist=0.1,
    random_state=42
)
embedding = reducer.fit_transform(data_pca)

# Plot
plt.figure(figsize=(12, 10))
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    alpha=0.5,
    c='blue'
)

plt.title('UMAP visualization of gene representations (PCA -> UMAP)')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')
sns.despine()
plt.tight_layout()
plt.show()

: 

: 

: 

In [ ]:
model.